## Assignment 2 Yuming Liu

### Problem 1(a)

In [1]:
import numpy as np
import pandas as pd
import timeit
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from dask import compute, delayed
import dask.multiprocessing
import multiprocessing
import error

In [2]:
df = pd.read_csv('data/Auto.csv', na_values = "?")
df.dropna(inplace=True)

In [3]:
df['orgn1'] = np.where(df['origin'] == 1, 1, 0)
df['orgn2'] = np.where(df['origin'] == 2, 1, 0)

In [4]:
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,orgn1,orgn2
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,1,0
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,1,0
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,1,0
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,1,0
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,1,0


In [5]:
df['mpg_high'] = np.where(df['mpg'] >= np.median(df['mpg']), 1, 0)

In [6]:
y = df['mpg_high'].values
X = df[['cylinders', 'displacement', 'horsepower', 'weight','acceleration', 'year', 'orgn1', 'orgn2']].values

t = timeit.default_timer()
error_rate = []

for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.65, test_size = 0.35, random_state = i)
    LogReg = LogisticRegression(n_jobs = 1, max_iter = 10000)
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    mse = np.mean((y_pred-y_test)**2)
    error_rate.append(mse)

print('The average error rate is', np.mean(error_rate))
elapsed_time = timeit.default_timer() - t
print('Elapsed time=', elapsed_time, 'seconds')

The average error rate is 0.10079710144927535
Elapsed time= 3.5749823999999997 seconds


### Problem 1(b)

In [7]:
num_cores = multiprocessing.cpu_count()
print('Number of available cores is', num_cores)

Number of available cores is 8


In [8]:
#def error(bootstrap, seed, X, y):
    #X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.65, test_size = 0.35, random_state = seed)
    #LogReg1 = LogisticRegression(n_jobs = 1, max_iter = 10000)
    #LogReg1.fit(X_train, y_train)
    #y_pred = LogReg1.predict(X_test)
    #mse = np.mean((y_pred-y_test)**2)
    
    #return mse

In [9]:
error_rate2 = []
t = timeit.default_timer()

for i in range(100):
    error_rate2.append(delayed(error.error)(i, i, X, y))

results = np.array(compute(*error_rate2, scheduler=dask.multiprocessing.get, num_workers=num_cores))

print('The error rate from parallel computing is', np.mean(results))

elapsed_time = timeit.default_timer() - t
print('Elapsed time=', elapsed_time, 'seconds')

The error rate from parallel computing is 0.10079710144927535
Elapsed time= 3.038628 seconds
